# Our Quantum Simulator
## Group: Brandon, Chris, Jacob, Michael

In [52]:
import gates
import utilities
import random
from IPython.display import clear_output


In [53]:
class QuantumProgram():
    def __init__(self,the_number_of_qubits):
        self.qubits = gates.zeroQubit()
        self.qubit_number = the_number_of_qubits
        self.operations = []
        self.classical = []
        for i in range(1, the_number_of_qubits):
            self.qubits = utilities.tensor(self.qubits,gates.zeroQubit())
    def getQubits(self):
        return self.qubits
    def addQubits(self, the_number_of_qubits):
        self.qubit_number += the_number_of_qubits
        for i in range(0, the_number_of_qubits):
            self.qubits = utilities.tensor(self.qubits,gates.zeroQubit())
    def getOperations(self):
        return self.operations
    def getNumberOfOperations(self):
        return len(self.operations)
    #TODO FIX measurement 
    def getQubit(self,position):
        return [self.qubits[0][position*2],self.qubits[0][position*2+1]]
    def getClassical(self, position):
        return self.classical[position]
    def measureQubit(self,qubitPosition,classicalRegister=None):
        qubit = self.getQubit(qubitPosition)
        zeroRange = qubit[0] * 100
        random.seed()
        measurement = random.random()*100
        if measurement <= zeroRange:
            measurement = 0
        else:
            measurement = 1
        if classicalRegister == None:
            self.classical.append(measurement)
        else:
            self.classical[classicalRegister] = measurement
        return measurement
    def measureAll(self):
        self.classical = []
        for i in range(0, self.qubit_number):
            self.measureQubit(i)
        return self.classical
    def measureAllAsString(self):
        self.measureAll()
        outString = ""
        for bit in self.classical:
            outString += str(bit)
        return outString
        
    def gateMaker(self,position_of_qubit,gateFunction):
        (self.qubits,returnedGate)=gates.apply_gate(gateFunction,self.qubits,position_of_qubit)
        self.operations.append(returnedGate)
        
    def Control(self,gateFunction,position_of_qubit,controls=[]):#Gate function being one of the above gates
        mat2=[]
        gateReturn = []
        mat2,gateReturn=gates.apply_gate(gateFunction,self.qubits,position_of_qubit,controls)
        self.qubits = mat2
        self.operations.append(gateReturn)
        return mat2
    def ControlClassic(self,gateFunction,position_of_qubit,controls=[],theta=None):#Gate function being one of the above gates
        mat2=[]
        gateReturn = []
        if 0 not in controls:
            if theta != None:#Not sure if overriding the given gateFunction with the one from gates.py works, but everything else should.
                mat2,gateReturn=gates.apply_gate(gateFunction, self.qubits,position_of_qubit,controls)
            else: 
                mat2,gateReturn=gates.apply_gate(gateFunction,self.qubits,position_of_qubit,controls)
        else:
            mat2,gateReturn=gates.apply_gate(gates.I(),self.qubits,position_of_qubit)
        self.qubits = mat2
        self.operations.append(gateReturn)
        return mat2

    def read_unitary(self):
        resultant = self.operations[0]
        first = True
        for operation in self.operations:
            if first:
                first = False
                continue
            else:
                resultant = utilities.matmul(resultant,operation)
        return resultant
    def read_state(self):
        return self.qubits
    def observing_probabilities(self):
        probabilities = []
        for i in range(0, self.qubit_number):
            probabilities.append(self.getQubit(i)*self.getQubit(i))
        return probabilities
    def execute(self,the_number_of_shots):
        executionDictionary = {}
        
        for string in utilities.getBinaryStrings(self.qubit_number):
            executionDictionary.setdefault(string,0)
        
        for i in range(0,the_number_of_shots):
            executionDictionary[self.measureAllAsString()] += 1  
        return executionDictionary
            
        

In [54]:
qc = QuantumProgram(3)
test = qc.getQubits()
print(gates.zeroQubit())
qc.gateMaker(2,gates.H())


system = qc.getQubits()
print(system)
results = qc.execute(50)
print(results)

[[1, 0]]
[[0.7071067811865476, 0.7071067811865476, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
{'000': 0, '001': 0, '010': 0, '011': 30, '100': 0, '101': 0, '110': 0, '111': 20}
